# Automatic Labeling with LDA and Genism (Interpreting a topic/label from keywords)

In [1]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jumananadir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Genism and Spacy imports

In [3]:
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
%matplotlib inline
import re

In [4]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
df = pd.read_csv('/Users/jumananadir/Downloads/Eluvio_DS_Challenge.csv')

In [7]:
data = df.title.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Scores killed in Pakistan clashes']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

print(data_words[:1])

[['scores', 'killed', 'in', 'pakistan', 'clashes']]


In [9]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

In [10]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [11]:
print(trigram_mod[bigram_mod[data_words[0]]])

['scores', 'killed', 'in', 'pakistan', 'clashes']


# Removing Stopwords, Lemmatizing and making Bigrams

In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [14]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [15]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
data_words_nostops = remove_stopwords(data_words)

In [17]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [19]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [20]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['score', 'kill', 'clash']]


In [21]:
id2word = corpora.Dictionary(data_lemmatized)

In [22]:
texts = data_lemmatized

In [23]:
#Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


# Checking word corresponding to an id

In [24]:
id2word[0]

'clash'

In [25]:
#Readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('clash', 1), ('kill', 1), ('score', 1)]]

# Latent Dirichlet Allocation (LDA) Model

In [26]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Keywords in the 10 topics/labels

In [27]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.209*"make" + 0.093*"start" + 0.086*"send" + 0.085*"life" + 0.083*"jail" + '
  '0.080*"try" + 0.052*"plant" + 0.048*"illegal" + 0.048*"hospital" + '
  '0.035*"unveil"'),
 (1,
  '0.289*"report" + 0.211*"protest" + 0.135*"border" + 0.128*"bomb" + '
  '0.063*"investigate" + 0.050*"secret" + 0.027*"gun" + 0.025*"flight" + '
  '0.008*"footage" + 0.000*"police"'),
 (2,
  '0.316*"military" + 0.139*"case" + 0.132*"win" + 0.114*"top" + 0.104*"town" '
  '+ 0.060*"violence" + 0.049*"envoy" + 0.014*"defy" + 0.000*"police" + '
  '0.000*"isis"'),
 (3,
  '0.449*"call" + 0.121*"reject" + 0.101*"old" + 0.062*"ally" + 0.057*"job" + '
  '0.052*"bid" + 0.042*"expose" + 0.025*"warning" + 0.000*"police" + '
  '0.000*"isis"'),
 (4,
  '0.267*"government" + 0.227*"russian" + 0.119*"back" + 0.094*"help" + '
  '0.089*"large" + 0.076*"build" + 0.065*"pay" + 0.019*"reactor" + '
  '0.000*"police" + 0.000*"isis"'),
 (5,
  '0.618*"attack" + 0.147*"opposition" + 0.067*"confirm" + 0.038*"poor" + '
  '0.029*"te

# Perplexity is a measure of how good a model is, lower perplexity is better

In [28]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -25.449046360522864

Coherence Score:  0.16816801400415524


In [29]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/jumananadir/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.271812  0.358463       1        1  10.617078
10     0.167663 -0.153512       2        1   9.472642
17     0.188396 -0.260854       3        1   8.732889
7      0.082906 -0.029529       4        1   6.950299
4      0.095289 -0.037600       5        1   6.895421
8      0.081571 -0.028749       6        1   6.563655
0     -0.010584  0.003806       7        1   5.429618
16    -0.009552  0.003585       8        1   5.273451
1     -0.010989  0.003842       9        1   5.244036
11     0.025416 -0.005246      10        1   5.225643
5     -0.053558  0.011140      11        1   4.267821
2     -0.036192  0.008508      12        1   4.159209
6     -0.069701  0.013251      13        1   3.383232
18    -0.056561  0.011581      14        1   3.265124
14    -0.092883  0.015725      15        1   3.051481
3     -0.079955  0.014422      16        1   3.032078
19    -0.110718  0.017220      17        1   2.663365
15    -0.103666  0.016681      18        1   2.533899
12    -0.133955  0.018535      19        1   1.929047
13    -0.144738  0.018729      20        1   1.310012, topic_info=      Category           Freq      Term          Total  loglift  logprob
73     Default  235232.000000       say  235232.000000  30.0000  30.0000
1      Default  123313.000000      kill  123313.000000  29.0000  29.0000
127    Default   83938.000000    attack   83938.000000  28.0000  28.0000
49     Default   75899.000000      year   75899.000000  27.0000  27.0000
97     Default   52793.000000     force   52793.000000  26.0000  26.0000
...        ...            ...       ...            ...      ...      ...
436    Topic20       0.165874     state       4.277010   1.0854 -12.4346
25618  Topic20       0.165892      isis       4.282303   1.0842 -12.4345
597    Topic20       0.165873      find       4.278185   1.0851 -12.4346
2107   Topic20       0.165863  militant       4.275220   1.0857 -12.4347
3901   Topic20       0.165864    syrian       4.277648   1.0851 -12.4347

[1051 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
129      14  0.999416     access
50       20  0.996688         ad
196       4  0.999719      allow
105      16  0.999281       ally
154       2  0.999049       also
...     ...       ...        ...
48        2  0.999615     worker
185       3  0.999915      world
79       18  0.998936      worry
49        2  0.999948       year
218       2  0.996869  yesterday

[198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 11, 18, 8, 5, 9, 1, 17, 2, 12, 6, 3, 7, 19, 15, 4, 20, 16, 13, 14])